# SAVEE and ESD testing

Here the best model found on the val subset of SAVEE is tested firstly on the test subset of SAVEE and also on the ESD test subdataset.

In addition, the reverse control is also done and best ESD trained model is tested on SAVEE test data.

In [1]:
from tensorflow import keras

import libs.data_loader as data_loader
import libs.data_operations as data_ops

import re

ESD_data_dir = "./data/ESD_data/"
SAVEE_data_dir = "./data/SAVEE/"

# SAVEE - specific
def get_SAVEE_label(file_path):
  parts = re.sub('.+\_|[0-9]+.wav', '', file_path)
  return parts

def get_SAVEE_speaker_name(file_path):
  parts = re.sub('.*[/]+|\_|[a-z]+[0-9]+.wav', '', file_path)
  return parts

# ESD - specific
def get_ESD_label(file_path):
  parts = re.sub('.*[/]+[a-z]\_|\_[0-9]+.wav', '', file_path)
  return parts

def get_ESD_speaker_name(file_path):
  parts = re.sub('.*[/]+|\_|[a-z]+_[0-9]+.wav', '', file_path)
  return parts

In [17]:
SAVEE_df, _, SAVEE_max_sample_rate = data_loader.get_dataset_information(SAVEE_data_dir, get_SAVEE_label, get_SAVEE_speaker_name)

In [18]:
ESD_df, _, ESD_max_sample_rate = data_loader.get_dataset_information(ESD_data_dir, get_ESD_label, get_ESD_speaker_name)

In [19]:
SAVEE_max_sample_rate, ESD_max_sample_rate

(44100, 16000)

## Best SAVEE on ESD

## Best ESD on SAVEE

Load ESD and SAVEE test sub dataset (also apply the same preprocessing needed by the model).

In this specific case, the best model trained on ESD data requires the split of data audio in 4 seconds and the application of crop and normalization.

In [39]:
# 'crop_norm'
data_preprocessing = lambda _: [
        data_ops.Crop(),
        data_ops.Normalize()]

# data_preprocessing_for_SAVEE = lambda _: [ 
#         data_ops.Crop(),
#         data_ops.Normalize()
#         ]

In [21]:
# remove from SAVEE label f and d because they'r not present as label in ESD
SAVEE_df = SAVEE_df.drop(columns=["label_f", "label_d"])

SAVEE_df = SAVEE_df.loc[SAVEE_df["label"] != "f"]
SAVEE_df = SAVEE_df.loc[SAVEE_df["label"] != "d"]

In [23]:
SAVEE_df["label"].unique()

array(['a', 'h', 'n', 'sa', 'su'], dtype=object)

In [40]:
# test ds tensor dataset of tuples (feats, target)
_, _, ESD_test_ds, _ = data_loader.load_datasets(ESD_df, ESD_max_sample_rate, 4, data_preprocessing, [70, 15, 15])

In [25]:
ESD_test_ds

<ParallelMapDataset element_spec=(TensorSpec(shape=(64000, 1), dtype=tf.float32, name=None), TensorSpec(shape=(5,), dtype=tf.uint8, name=None))>

In [41]:
_, _, SAVEE_test_ds, _ = data_loader.load_datasets(SAVEE_df, ESD_max_sample_rate, 4, data_preprocessing, [70, 15, 15])

In [42]:
SAVEE_test_ds

<ParallelMapDataset element_spec=(TensorSpec(shape=(64000, 1), dtype=tf.float32, name=None), TensorSpec(shape=(5,), dtype=tf.uint8, name=None))>

In [43]:
ESD_test_ds = ESD_test_ds.batch(64)
SAVEE_test_ds = SAVEE_test_ds.batch(64)

In [44]:
#  model_name = f"m{model_factory.get_model_name()}_s{audio_seconds}_b{batch_size}_d{dropout}_p{patience}_o_{data_ops_name}_sz{str(train_val_tests_percentage).replace(' ', '')[1:-1]}"
ESD_model = keras.models.load_model("./saved_models/mModel_convl5_fcl3_fcsize20_filters32_fsize9_poolsize4_act_relu_drop0.2_lr001_b19_b2999____s4_b64_d0.5_p80_o_crop_norm_sz70,15,15/")
ESD_model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_85 (Conv1D)          (None, 63992, 32)         320       
                                                                 
 batch_normalization_85 (Bat  (None, 63992, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling1d_68 (MaxPoolin  (None, 15998, 32)        0         
 g1D)                                                            
                                                                 
 conv1d_86 (Conv1D)          (None, 15990, 32)         9248      
                                                                 
 batch_normalization_86 (Bat  (None, 15990, 32)        128       
 chNormalization)                                                
                                                     

In [45]:
ESD_model.evaluate(ESD_test_ds)

41/41 [==============================] - 32s 683ms/step - loss: 2.0560 - accuracy: 0.7172


[2.055967092514038, 0.7171755433082581]

In [46]:
ESD_model.evaluate(SAVEE_test_ds)

1/1 [==============================] - 3s 3s/step - loss: 41.9087 - accuracy: 0.1964


[41.90869140625, 0.1964285671710968]